In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K

Using TensorFlow backend.


In [2]:
steps = 5
height = 108
width = 108

In [3]:
files = glob.glob('E:/notes_database/index/*')

In [28]:
def main(optimizer):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2)(layers)
    layers = Conv2D(32, (3,3), border_mode='same', activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2)(layers)
    layers = Conv2D(64, (3,3), border_mode='same', activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2)(layers)
    layers = Conv2D(64, (3,3), border_mode='same', activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2)(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['binary_accuracy'])
    return model

In [13]:
kf = KFold(n_splits=5, random_state=7)

In [14]:
train_order = []
test_order = []
files = np.array(files)
for x,y in kf.split(files):
    train_order.append(files[x])
    test_order.append(files[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [15]:
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for i in range(prediction.shape[0]):
        for j in range(prediction.shape[1]):
            if abs(prediction[i][j] - y_test[i][j]) > 0.1:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [16]:
import time

In [29]:
def train_test(num, name, epochs=100, batchsize=32, split=3, optimizer='adam'):
    classifier = main(optimizer)
    train_nums = []
    length = train_order[num].shape[0]//split
    for i in range(split-1):
        train_nums.append(train_order[num][(i)*length:(i+1)*length])
    train_nums.append(train_order[num][(split-1)*length:])
        
    for order in range(split):
        for epoch in range(epochs):
            
            #the first train_number works as the template
            lstm_train, cnn_train, y_train = prepare_inputs(train_nums[order][0])
            for file_name in train_nums[order][1:]:
                lstm_input, cnn_input, y_train_input = prepare_inputs(file_name)
                lstm_train = np.append(lstm_train, lstm_input, axis=0)
                cnn_train = np.append(cnn_train, cnn_input, axis=0)
                y_train = np.append(y_train, y_train_input, axis=0)
            cnn_train = cnn_train.reshape((cnn_train.shape[0], 108, 108, 1))
                
            classifier.fit([cnn_train, lstm_train], y_train, epochs=1, batch_size=batchsize, verbose=1,
                          validation_data=([cnn_test, lstm_test], y_test))
            #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
            #print('val_loss: ', CV_loss)
            #print('val_accuarcy', accuracy)
    classifier.save_weights(name, overwrite=True)
    del classifier
    K.clear_session()

In [131]:
train_test(0, 'weight-notes-1.hdf5')
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
  del sys.path[0]


138.18120694160461
Train on 27780 samples, validate on 20929 samples
Epoch 1/1
27780/27780 [==============================] - 39s 1ms/step - loss: 0.0914 - acc: 0.3977 - val_loss: 0.0616 - val_acc: 0.5290
Train on 27780 samples, validate on 20929 samples
Epoch 1/1
27780/27780 [==============================] - 33s 1ms/step - loss: 0.0544 - acc: 0.4889 - val_loss: 0.0466 - val_acc: 0.4891
Train on 27780 samples, validate on 20929 samples
Epoch 1/1
27780/27780 [==============================] - 32s 1ms/step - loss: 0.0457 - acc: 0.4847 - val_loss: 0.0407 - val_acc: 0.4765
Train on 27780 samples, validate on 20929 samples
Epoch 1/1
27780/27780 [==============================] - 32s 1ms/step - loss: 0.0419 - acc: 0.4801 - val_loss: 0.0454 - val_acc: 0.4688
Train on 27780 samples, validate on 20929 samples
Epoch 1/1
27780/27780 [==============================] - 32s 1ms/step - loss: 0.0395 - acc: 0.4793 - val_loss: 0.0421 - val_acc: 0.5200
Train on 27780 samples, validate on 20929 samples
E

KeyboardInterrupt: 

In [26]:
def train_test_whole(num, name, epochs=100, batchsize=32, optimizer='adam'):
    classifier = main(optimizer)
        
    #the first train_number works as the template
    count = 1
    ce = 0
    lstm_train, cnn_train, y_train = prepare_inputs(train_order[num][0])
    for file_name in train_order[num][1:]:
        count+=1
        if count == ce + 100:
            print(count)
            ce = count
        lstm_input, cnn_input, y_train_input = prepare_inputs(file_name)
        lstm_train = np.append(lstm_train, lstm_input, axis=0)
        cnn_train = np.append(cnn_train, cnn_input, axis=0)
        y_train = np.append(y_train, y_train_input, axis=0)
    cnn_train = cnn_train.reshape((cnn_train.shape[0], 108, 108, 1))
                
    classifier.fit([cnn_train, lstm_train], y_train, epochs=epochs, batch_size=batchsize, verbose=1)
    #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
    #print('val_loss: ', CV_loss)
    #print('val_accuarcy', accuracy)
    classifier.save_weights(name, overwrite=True)
    del classifier
    K.clear_session()

In [18]:
kf = KFold(n_splits=3, random_state=7)
train_order = []
test_order = []
files = np.array(files)[:900]
for x,y in kf.split(files):
    train_order.append(files[x])
    test_order.append(files[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [23]:
train_test_whole(0, 'weight-notes-1.hdf5')
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]


100
200
300
400
500
600
Epoch 1/100
62698/62698 [==============================] - 63s 1ms/step - loss: 0.0689 - acc: 0.4481
Epoch 2/100
62698/62698 [==============================] - 58s 918us/step - loss: 0.0428 - acc: 0.4838TA: 0s - loss: 0.04
Epoch 3/100
62698/62698 [==============================] - 57s 917us/step - loss: 0.0383 - acc: 0.4805
Epoch 4/100
62698/62698 [==============================] - 58s 918us/step - loss: 0.0361 - acc: 0.4820
Epoch 5/100
62698/62698 [==============================] - 58s 917us/step - loss: 0.0347 - acc: 0.4723
Epoch 6/100
62698/62698 [==============================] - 58s 918us/step - loss: 0.0336 - acc: 0.4755
Epoch 7/100
62698/62698 [==============================] - 58s 919us/step - loss: 0.0326 - acc: 0.4757
Epoch 8/100
62698/62698 [==============================] - 58s 925us/step - loss: 0.0321 - acc: 0.4676
Epoch 9/100
62698/62698 [==============================] - 58s 925us/step - loss: 0.0315 - acc: 0.47250s - loss: 0.0315 
Epoch 10/100
6

56497

In [30]:
adam = keras.optimizers.Adam(lr=0.01)
train_test_whole(0, 'weight-notes-2.hdf5', optimizer=adam)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]


100
200
300
400
500
600
Epoch 1/100
62698/62698 [==============================] - 59s 945us/step - loss: 0.0754 - acc: 0.4544
Epoch 2/100
62698/62698 [==============================] - 57s 915us/step - loss: 0.0514 - acc: 0.5000
Epoch 3/100
62698/62698 [==============================] - 57s 914us/step - loss: 0.0449 - acc: 0.4936
Epoch 4/100
62698/62698 [==============================] - 57s 914us/step - loss: 0.0425 - acc: 0.4878
Epoch 5/100
62698/62698 [==============================] - 57s 915us/step - loss: 0.0408 - acc: 0.4914
Epoch 6/100
62698/62698 [==============================] - 57s 916us/step - loss: 0.0398 - acc: 0.4952
Epoch 7/100
62698/62698 [==============================] - 57s 914us/step - loss: 0.0388 - acc: 0.5016
Epoch 8/100
62698/62698 [==============================] - 57s 915us/step - loss: 0.0383 - acc: 0.50081s - loss: 
Epoch 9/100
62698/62698 [==============================] - 57s 915us/step - loss: 0.0379 - acc: 0.5013
Epoch 10/100
62698/62698 [============

105

In [ ]:
adam = keras.optimizers.Adam(lr=0.01)
train_test_whole(0, 'weight-notes-200.hdf5', optimizer=adam)
gc.collect()

In [137]:
train_test_whole(0, 'weight-notes-1.hdf5')
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
  del sys.path[0]


135.32289028167725
100
200
300
400
500
600


KeyboardInterrupt: 